In [ ]:
import panel as pn
pn.extension()

The ``FileInput`` widget allows uploading one or more file from the frontend and makes the filename, file data and MIME type available in Python.

For more information about listening to widget events and laying out widgets refer to the [widgets user guide](../../user_guide/Widgets.ipynb). Alternatively you can learn how to build GUIs by declaring parameters independently of any specific widgets in the [param user guide](../../user_guide/Param.ipynb). To express interactivity entirely using Javascript without the need for a Python server take a look at the [links user guide](../../user_guide/Param.ipynb).

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

##### Core

* **``accept``** (str):  A list of file input filters that restrict what files the user can pick from
* **``filename``** (str/list): The filename(s) of the uploaded file(s)
* **``mime_type``** (str/list): The mime type(s) of the uploaded file(s)
* **``multiple``** (boolean): Whether to allow uploading multiple files
* **``value``** (bytes/list): A bytes object containing the file data or if `multiple` is set a list of bytes types.

___

In [ ]:
file_input = pn.widgets.FileInput()

file_input

To read out the content of the file you can access the ``value`` parameter, which holds a bytestring containing the file's contents. Additionally information about the file type is made available on the ``filetype`` parameter expressed as a MIME type, e.g. ``image/png`` or ``text/csv``.

In [ ]:
file_input.value

The widget also has a ``save`` method that allows saving the uploaded data to a file or BytesIO object.

In [ ]:
if file_input.value is not None:
    file_input.save('test.png')

The `accept` parameter restricts what files the user can pick from. This consists of comma-separated list of standard HTML
file input filters. Values can be:

* `<file extension>` - Specific file extension(s) (e.g: .gif, .jpg, .png, .doc) are pickable
* `audio/*` - all sound files are pickable
* `video/*` - all video files are pickable
* `image/*` - all image files are pickable
* `<media type>` - A valid [IANA Media Type](https://www.iana.org/assignments/media-types/media-types.xhtml), with no parameters.

In [ ]:
file_input = pn.widgets.FileInput(accept='.csv,.json')

file_input

To allow uploading multiple files we can also set `multiple=True`:

In [ ]:
file_input = pn.widgets.FileInput(accept='.png', multiple=True)

file_input

When uploading one or more files the `filename`, `mime_type` and `value` parameters will now be lists. 

### Controls

The `FileInput` widget exposes a number of options which can be changed from both Python and Javascript. Try out the effect of these parameters interactively:

In [ ]:
pn.Row(file_input.controls(jslink=True), file_input)